In [ ]:
# testing uncentered-trained RNN 
# importing pytorch
import torch
print(torch.__version__)


In [ ]:
# running data config file
%run '/home/christianl/Zhang-Lab/Zhang Lab Code/Boilerplate/model_boilerplate_remote.py'

In [ ]:
# specifying testing data
net = pd.read_csv('/home/christianl/Zhang-Lab/Zhang Lab Data/Full data files/network(full).tsv', sep = '\t')
x_in = x_test
y_out = y_test

In [1]:
"""
SUPER SIMPLE LOADER - One Function Does Everything
===================================================

Just call load_model() and you're done!
"""

import torch
import pandas as pd
import numpy as np
import sys

# Update this path to your LEMBAS directory
sys.path.append('/home/christian/Zhang-Lab/RNN')
from bionetwork.py import SignalingModel


def load_model(checkpoint_path, node_names=None, net=None, X_in=None, y_out=None):
    """
    Load your model in one line!
    
    Parameters
    ----------
    checkpoint_path : str
        Path to your .pt checkpoint file
    node_names : list, optional
        List of node names. If None, uses generic names.
    net : pd.DataFrame, optional
        Network topology. If None, reconstructs from checkpoint.
    X_in : pd.DataFrame, optional
        Input data. If None, creates dummy data.
    y_out : pd.DataFrame, optional
        Output data. If None, creates dummy data.
        
    Returns
    -------
    model : SignalingModel
        Your loaded model, ready to use!
        
    Example
    -------
    >>> model = load_model('signaling_model.v1.pt')
    >>> # That's it! Now use it:
    >>> model.eval()
    >>> predictions, _ = model(model.X_in[:5])
    """
    
    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    state_dict = checkpoint['state_dict']
    config = {
        'seed': checkpoint.get('seed', 888),
        'bionet_params': checkpoint.get('bionet_params'),
        'projection_amplitude_in': checkpoint.get('projection_amplitude_in', 1.0),
        'projection_amplitude_out': checkpoint.get('projection_amplitude_out', 1.0)
    }
    
    # Get dimensions from state_dict
    network_key = [k for k in state_dict.keys() if 'network' in k.lower() and 'weight' in k.lower() and len(state_dict[k].shape) == 2][0]
    adj_matrix = state_dict[network_key].cpu().numpy()
    n_nodes = adj_matrix.shape[0]
    
    input_key = [k for k in state_dict.keys() if 'input' in k.lower() and 'weight' in k.lower()][0]
    n_ligands = state_dict[input_key].shape[0]
    
    output_key = [k for k in state_dict.keys() if 'output' in k.lower() and 'weight' in k.lower()][0]
    n_tfs = state_dict[output_key].shape[0]
    
    # Create node names if not provided
    if node_names is None:
        node_names = [f'node_{i}' for i in range(n_nodes)]
    
    # Reconstruct network if not provided
    if net is None:
        sources, targets, weights = [], [], []
        for i in range(n_nodes):
            for j in range(n_nodes):
                if adj_matrix[i, j] != 0:
                    targets.append(node_names[i])
                    sources.append(node_names[j])
                    weights.append(adj_matrix[i, j])
        
        net = pd.DataFrame({
            'source': sources,
            'target': targets,
            'mode_of_action': [1 if w > 0 else -1 for w in weights]
        })
    
    # Create dummy data if not provided
    if X_in is None:
        X_in = pd.DataFrame(
            np.random.rand(100, n_ligands) * 0.1,
            columns=[node_names[i] for i in range(n_ligands)]
        )
    
    if y_out is None:
        y_out = pd.DataFrame(
            np.random.rand(100, n_tfs) * 0.1,
            columns=[node_names[i] for i in range(n_tfs)]
        )
    
    # Initialize model
    model = SignalingModel(
        net=net,
        X_in=X_in,
        y_out=y_out,
        projection_amplitude_in=config['projection_amplitude_in'],
        projection_amplitude_out=config['projection_amplitude_out'],
        bionet_params=config['bionet_params'],
        activation_function='MML',
        dtype=torch.float32,
        device='cpu',
        seed=config['seed']
    )
    
    # Load weights
    model.load_state_dict(state_dict)
    
    print("✅ Model loaded successfully!")
    print(f"   Nodes: {n_nodes}, Ligands: {n_ligands}, TFs: {n_tfs}")
    
    return model


# =============================================================================
# USAGE EXAMPLES
# =============================================================================

if __name__ == "__main__":
    
    # Example 1: Load with everything automatic (dummy data)
    print("Example 1: Automatic loading with dummy data")
    model = load_model('/home/christian/Zhang-Lab/Zhang Lab Data/Saved models/RNN/signaling_model.v1.pt')
    
    # Test it
    model.eval()
    with torch.no_grad():
        y_pred, _ = model(model.X_in[:3])
        print(f"Prediction shape: {y_pred.shape}")
        print(f"Sample value: {y_pred[0, 0].item():.6f}")
    
    print("\n" + "="*70)
    
    # Example 2: Load with your own node names
    print("\nExample 2: With node names")
    # node_names = ['EGF', 'TGFA', 'FGF1', ...]  # Your names here
    # model = load_model('signaling_model.v1.pt', node_names=node_names)
    
    # Example 3: Load with your own network and data
    print("\nExample 3: With real data")
    # net = pd.read_csv('your_network.csv')
    # X_in = pd.read_csv('your_X_in.csv', index_col=0)
    # y_out = pd.read_csv('your_y_out.csv', index_col=0)
    # model = load_model('signaling_model.v1.pt', net=net, X_in=X_in, y_out=y_out)
    
    print("\n✅ Done! Your model is ready to use.")

ModuleNotFoundError: No module named 'bionetwork.py'; 'bionetwork' is not a package

In [7]:
analyze_what_you_have('/home/christianl/Zhang-Lab/Zhang Lab Data/Saved models/RNN/signaling_model.v1.pt')

STATE_DICT ANALYSIS


/tmp/ipykernel_2640175/2818158888.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(state_dict_path, map_location='cpu')


KeyError: 'input_layer.weights'

In [ ]:
# initialization parameters
projection_amplitude_in = 1  # default of 1 
projection_amplitude_out = ???  # equal to the number of nodes in the final fixed network 
activation_function = 'MML'
bionet_params = {
    'target_steps': 100,
    'max_steps': 300,      # RNN iterations
    'exp_factor': 20,
    'leak': 0.01,          # for activation function
    'tolerance': 1e-5,     # convergence threshold
    'spectral_target': 1e-7
}
ban_list = None

dtype = torch.dtype=torch.float32
device = 'cpu' # was trained on gpu 
seed = 888 # default of 888 by Kejun's self.setseeds function unless specifcied




In [ ]:
# loading in uncentered data trained model learned weighted
uncentered_data_RNN_weights = torch.load('/home/christianl/Zhang-Lab/Zhang Lab Data/Saved models/RNN/signaling_model.v1.pt')

In [ ]:
print("=== Model Parameters ===")
print(f"Input shape: {trained_model.X_in.shape}")
print(f"Output shape: {trained_model.y_out.shape}")
print(f"Number of nodes: {len(trained_model.node_idx_map)}")
print(f"Projection amplitude in: {trained_model.input_layer.projection_amplitude}")
print(f"Projection amplitude out: {trained_model.projection_amplitude_out}")
print(f"Device: {trained_model.device}")
print(f"Seed: {trained_model.seed}")
print(f"\nBioNet params: {trained_model.signaling_network.training_params}")

In [ ]:
# getting predicted values y_pred from test set 
rnn_y_pred = loaded_model.predict(x_test) 
print(type(rnn_y_pred), rnn_y_pred.shape)